## AUTOMATED LEDGER BALANCING.

Below is a self-contained Python script that can balance a simple ledger automatically.
It reads a CSV file of transactions, computes the running balance, and flags any line where the calculated balance does not match the “expected balance” column (if you provide one).
You can run it with python balance_ledger.py ledger.csv.

In [ ]:
#!/usr/bin/env python3
"""
balance_ledger.py

Automate ledger balancing.

Features
--------
* Reads a CSV with columns: Date, Description, Debit, Credit, [Balance]
* Supports optional starting balance.
* Computes running balance.
* Reports mismatches (if a Balance column is present).
* Writes a clean, balanced ledger to `balanced_ledger.csv`.
"""

import argparse
import csv
import sys
from decimal import Decimal, ROUND_HALF_UP
from pathlib import Path

# ----------------------------------------------------------------------
# Helper functions
# ----------------------------------------------------------------------
def parse_amount(s: str) -> Decimal:
    """Convert a string (e.g. '1,234.56' or '-567.00') to Decimal."""
    if not s:
        return Decimal('0')
    # Remove commas and any whitespace
    cleaned = s.replace(',', '').strip()
    return Decimal(cleaned).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)


def read_ledger(csv_path: Path, start_balance: Decimal = Decimal('0')) -> list[dict]:
    """
    Return a list of transaction dicts with a computed running balance.
    """
    rows = []
    running = start_balance

    with csv_path.open(newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        expected_fields = {'Date', 'Description', 'Debit', 'Credit'}
        missing = expected_fields - set(reader.fieldnames or [])
        if missing:
            raise ValueError(f"CSV missing required columns: {', '.join(missing)}")

        for i, row in enumerate(reader, start=2):  # start=2 → line number in file
            try:
                debit = parse_amount(row.get('Debit', ''))
                credit = parse_amount(row.get('Credit', ''))
            except Exception as e:
                print(f"Line {i}: Invalid amount – {e}", file=sys.stderr)
                continue

            net = credit - debit
            running += net

            record = {
                'line': i,
                'date': row['Date'].strip(),
                'description': row['Description'].strip(),
                'debit': debit,
                'credit': credit,
                'net': net,
                'balance': running,
                'expected_balance': None,
            }

            # Optional: user-provided balance column
            if 'Balance' in row:
                try:
                    record['expected_balance'] = parse_amount(row['Balance'])
                except Exception:
                    print(f"Line {i}: Invalid Balance value", file=sys.stderr)

            rows.append(record)

    return rows


def write_balanced(csv_path: Path, records: list[dict]):
    """Write a clean CSV with computed balances."""
    fieldnames = [
        'Date', 'Description', 'Debit', 'Credit', 'Balance'
    ]
    with csv_path.open('w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for r in records:
            writer.writerow({
                'Date': r['date'],
                'Description': r['description'],
                'Debit': f"{r['debit']:,}",
                'Credit': f"{r['credit']:,}",
                'Balance': f"{r['balance']:,}",
            })


def report_mismatches(records: list[dict]):
    """Print any lines where computed != expected balance."""
    mismatches = [
        r for r in records
        if r['expected_balance'] is not None and r['balance'] != r['expected_balance']
    ]
    if not mismatches:
        print("All balances match!")
        return

    print("\nMISMATCHES FOUND:")
    print("-" * 80)
    print(f"{'Line':<6} {'Date':<12} {'Description':<30} {'Computed':<15} {'Expected':<15} {'Diff':<10}")
    print("-" * 80)
    for r in mismatches:
        diff = r['balance'] - r['expected_balance']
        print(f"{r['line']:<6} {r['date']:<12} {r['description']:<30} "
              f"{r['balance']:<15,} {r['expected_balance']:<15,} {diff:<+10,}")
    print("-" * 80)


# ----------------------------------------------------------------------
# CLI
# ----------------------------------------------------------------------
def main():
    parser = argparse.ArgumentParser(
        description="Automate ledger balancing from a CSV file."
    )
    parser.add_argument(
        "csv_file",
        type=Path,
        help="Path to the input ledger CSV."
    )
    parser.add_argument(
        "--start", "-s",
        type=Decimal,
        default=Decimal('0'),
        help="Starting balance (default: 0). Use commas if needed, e.g. 12,345.67"
    )
    parser.add_argument(
        "--output", "-o",
        type=Path,
        default=Path("balanced_ledger.csv"),
        help="Output CSV with computed balances (default: balanced_ledger.csv)"
    )
    args = parser.parse_args()

    if not args.csv_file.is_file():
        print(f"Error: File not found – {args.csv_file}", file=sys.stderr)
        sys.exit(1)

    try:
        records = read_ledger(args.csv_file, args.start)
    except Exception as e:
        print(f"Error reading ledger: {e}", file=sys.stderr)
        sys.exit(1)

    # Final balance
    final_balance = records[-1]['balance'] if records else args.start
    print(f"\nLedger processed – {len(records)} transactions")
    print(f"Final computed balance: {final_balance:,.2f}")

    # Report mismatches
    report_mismatches(records)

    # Write clean output
    write_balanced(args.output, records)
    print(f"\nBalanced ledger written to: {args.output.resolve()}")


if __name__ == "__main__":
    main()



#Date,Description,Debit,Credit,Balance
#2025-01-01,Opening balance,,12000.00,12000.00
#2025-01-05,Sale of maize,,8500.00,20500.00
#2025-01-10,Fertilizer purchase,3200.00,,17300.00
#2025-01-15,Transport cost,1500.00,,15800.00

## DATA VALIDATION.


In [ ]:
Here is an automated data validation script.

In [ ]:
#!/usr/bin/env python3
"""
balance_and_validate.py

* Balances a ledger (same as before)
* Validates every row against a flexible rule set
* Reports clean summary + detailed CSV of failures
"""

import argparse
import csv
import re
import sys
from decimal import Decimal, ROUND_HALF_UP
from pathlib import Path
from typing import Any, Dict, List, Optional

# ----------------------------------------------------------------------
# Core types
# ----------------------------------------------------------------------
Rule = Dict[str, Any]          # one validation rule
Record = Dict[str, Any]        # one ledger row (augmented)
ValidationResult = List[Dict[str, Any]]

# ----------------------------------------------------------------------
# Helper functions
# ----------------------------------------------------------------------
def parse_amount(s: str) -> Decimal:
    if not s:
        return Decimal('0')
    cleaned = s.replace(',', '').strip()
    return Decimal(cleaned).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)


def load_rules(csv_path: Path) -> List[Rule]:
    """Load validation rules from a CSV.

    Expected columns:
        column, type, param1, param2, message
    """
    rules: List[Rule] = []
    with csv_path.open(newline='', encoding='utf-8') as f:
        for row in csv.DictReader(f):
            if not row.get('column'):
                continue
            rules.append({
                'column': row['column'].strip(),
                'type': row['type'].strip().lower(),
                'param1': row.get('param1', '').strip(),
                'param2': row.get('param2', '').strip(),
                'message': row.get('message', '').strip() or f"Invalid {row['column']}",
            })
    return rules


def validate_record(rec: Record, rules: List[Rule], line: int) -> List[str]:
    """Return list of error messages for this record."""
    errs: List[str] = []

    for r in rules:
        col = r['column']
        val = rec.get(col)

        # ------------------------------------------------------------------
        # 1. Required field
        # ------------------------------------------------------------------
        if r['type'] == 'required' and (val is None or str(val).strip() == ''):
            errs.append(r['message'])
            continue

        if val is None:
            continue  # column not present → skip rule

        # ------------------------------------------------------------------
        # 2. Data-type checks
        # ------------------------------------------------------------------
        if r['type'] == 'date_iso':
            if not re.fullmatch(r'\d{4}-\d{2}-\d{2}', str(val)):
                errs.append(r['message'])

        elif r['type'] == 'positive':
            try:
                if Decimal(str(val)) < 0:
                    errs.append(r['message'])
            except Exception:
                errs.append(r['message'])

        elif r['type'] == 'regex':
            if not re.fullmatch(r['param1'], str(val)):
                errs.append(r['message'])

        elif r['type'] == 'in_list':
            allowed = {x.strip() for x in r['param1'].split('|') if x.strip()}
            if str(val).strip() not in allowed:
                errs.append(r['message'])

        elif r['type'] == 'range':
            try:
                v = Decimal(str(val))
                lo = Decimal(r['param1']) if r['param1'] else None
                hi = Decimal(r['param2']) if r['param2'] else None
                if lo is not None and v < lo:
                    errs.append(r['message'])
                if hi is not None and v > hi:
                    errs.append(r['message'])
            except Exception:
                errs.append(r['message'])

    return errs


# ----------------------------------------------------------------------
# Ledger processing
# ----------------------------------------------------------------------
def process_ledger(
    ledger_path: Path,
    start_balance: Decimal,
    rules: List[Rule],
    check_balance_column: bool,
) -> tuple[List[Record], ValidationResult, Decimal]:
    records: List[Record] = []
    failures: ValidationResult = []
    running = start_balance
    prev_date = None

    with ledger_path.open(newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        required = {'Date', 'Description', 'Debit', 'Credit'}
        missing = required - set(reader.fieldnames or {})
        if missing:
            raise ValueError(f"Missing columns: {', '.join(missing)}")

        for i, row in enumerate(reader, start=2):
            # ---- parse amounts -------------------------------------------------
            try:
                debit = parse_amount(row.get('Debit', ''))
                credit = parse_amount(row.get('Credit', ''))
            except Exception as e:
                failures.append({
                    'line': i,
                    'column': 'Debit/Credit',
                    'error': f"Bad amount – {e}",
                })
                continue

            net = credit - debit
            running += net

            # ---- build record --------------------------------------------------
            rec: Record = {
                'line': i,
                'date': row['Date'].strip(),
                'description': row['Description'].strip(),
                'debit': debit,
                'credit': credit,
                'net': net,
                'balance': running,
                'expected_balance': None,
            }
            for k in row:
                if k not in rec:
                    rec[k] = row[k].strip()

            # optional Balance column
            if check_balance_column and 'Balance' in row:
                try:
                    rec['expected_balance'] = parse_amount(row['Balance'])
                except Exception:
                    failures.append({
                        'line': i,
                        'column': 'Balance',
                        'error': 'Invalid number',
                    })

            # ---- date order ----------------------------------------------------
            if prev_date and rec['date'] < prev_date:
                failures.append({
                    'line': i,
                    'column': 'Date',
                    'error': f"Out of order (previous {prev_date})",
                })
            prev_date = rec['date']

            # ---- run custom rules ---------------------------------------------
            errs = validate_record(rec, rules, i)
            for msg in errs:
                failures.append({'line': i, 'column': r['column'], 'error': msg})

            records.append(rec)

    return records, failures, running


# ----------------------------------------------------------------------
# Output helpers
# ----------------------------------------------------------------------
def write_balanced(csv_path: Path, records: List[Record]):
    fields = ['Date', 'Description', 'Debit', 'Credit', 'Balance']
    with csv_path.open('w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fields)
        writer.writeheader()
        for r in records:
            writer.writerow({
                'Date': r['date'],
                'Description': r['description'],
                'Debit': f"{r['debit']:,}",
                'Credit': f"{r['credit']:,}",
                'Balance': f"{r['balance']:,}",
            })


def write_failures(csv_path: Path, failures: ValidationResult):
    fields = ['line', 'column', 'error']
    with csv_path.open('w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fields)
        writer.writeheader()
        writer.writerows(failures)


# ----------------------------------------------------------------------
# CLI
# ----------------------------------------------------------------------
def main():
    parser = argparse.ArgumentParser(
        description="Balance a ledger **and** validate every row."
    )
    parser.add_argument("ledger_csv", type=Path, help="Input ledger CSV")
    parser.add_argument(
        "--rules", "-r", type=Path, help="CSV with validation rules (see example below)"
    )
    parser.add_argument(
        "--start", "-s", type=Decimal, default=Decimal('0'),
        help="Opening balance (default 0). Use commas if needed."
    )
    parser.add_argument(
        "--output", "-o", type=Path, default=Path("balanced_ledger.csv"),
        help="Balanced ledger output"
    )
    parser.add_argument(
        "--failures", "-f", type=Path, default=Path("validation_failures.csv"),
        help="CSV with every validation error"
    )
    parser.add_argument(
        "--no-balance-check", action="store_false", dest="check_balance",
        help="Skip checking supplied Balance column"
    )
    args = parser.parse_args()

    # ------------------------------------------------------------------
    # Load rules (default set if none supplied)
    # ------------------------------------------------------------------
    default_rules: List[Rule] = [
        {'column': 'Date', 'type': 'date_iso', 'message': 'Date must be YYYY-MM-DD'},
        {'column': 'Description', 'type': 'required', 'message': 'Description missing'},
        {'column': 'Debit', 'type': 'positive', 'message': 'Debit cannot be negative'},
        {'column': 'Credit', 'type': 'positive', 'message': 'Credit cannot be negative'},
    ]

    if args.rules and args.rules.is_file():
        custom = load_rules(args.rules)
        rules = default_rules + custom
        print(f"Loaded {len(custom)} custom rule(s) from {args.rules}")
    else:
        rules = default_rules
        print("Using built-in default rules (add --rules for custom)")

    # ------------------------------------------------------------------
    # Process
    # ------------------------------------------------------------------
    records, failures, final_bal = process_ledger(
        args.ledger_csv, args.start, rules, args.check_balance
    )

    # ------------------------------------------------------------------
    # Report
    # ------------------------------------------------------------------
    print(f"\nProcessed {len(records)} transaction(s)")
    print(f"Final balance: {final_bal:,.2f}")

    if failures:
        print(f"\n{len(failures)} validation error(s) – see {args.failures}")
        write_failures(args.failures, failures)
    else:
        print("\nAll rows passed validation!")

    # Balance-column mismatches (separate from custom rules)
    balance_mismatches = [
        r for r in records
        if args.check_balance and r['expected_balance'] is not None
        and r['balance'] != r['expected_balance']
    ]
    if balance_mismatches:
        print("\nBalance column mismatches:")
        for r in balance_mismatches:
            diff = r['balance'] - r['expected_balance']
            print(f"  Line {r['line']}: computed {r['balance']:,} vs expected {r['expected_balance']:,} (Δ {diff:+,})")

    write_balanced(args.output, records)
    print(f"\nBalanced ledger → {args.output.resolve()}")


if __name__ == "__main__":
    main()